In [1]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv
from stable_baselines3 import SAC, HerReplayBuffer
from stable_baselines3.common.cmd_util import make_vec_env
from environments import *

import torch
import numpy as np
import gym
import imageio

c:\Users\ssuro\anaconda3\lib\site-packages\torch\utils\tensorboard\__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
c:\Users\ssuro\anaconda3\lib\site-packages\tensorboard\compat\proto\tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
c:\Users\ssuro\anaconda3\lib\site-packages\tensorboard\compat\proto\tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(
c:\Users\ssuro\anaconda3\lib\site-packages\tensorboard\compat\proto\tensor_shape_pb2

In [2]:
env_id = 'G1Goal-v0'
video_folder = 'renders'
video_length = 500

In [3]:
env = make_vec_env("G1Goal-v1", 1)

c:\Users\ssuro\anaconda3\lib\site-packages\gym\core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\ssuro\anaconda3\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [4]:
model = SAC(
    "MultiInputPolicy", 
    env, 
    verbose=1,
    replay_buffer_class=HerReplayBuffer,
    replay_buffer_kwargs=dict(
      n_sampled_goal=4,
      goal_selection_strategy="future",
      max_episode_length=1000,
      online_sampling=True,
    ),
    policy_kwargs=dict(
        net_arch=[512, 512, 512, 512], 
        activation_fn=torch.nn.ELU,
        n_critics=5,
    ),
    learning_rate=1e-4,
    learning_starts=1000,
    tensorboard_log="./logs/sac+her/"
)

Using cuda device


MemoryError: Unable to allocate 454. MiB for an array with shape (1000, 1000, 1, 119) and data type float32

In [ ]:
model = SAC.load("trained_models/g1-forward/sac+her", env=env)

In [ ]:
images = []

In [ ]:
obs = env.reset()
img = env.render(mode='rgb_array')

for i in range(video_length + 1):
  images.append(img)
  action = model.predict(obs)
  obs, _, _, _ = env.step(action[0])
  img = env.render(mode='rgb_array')

env.close()

In [ ]:
imageio.mimsave(
    'renders/sac.gif', 
    [np.array(img) for i, img in enumerate(images) if i%2 == 0], 
    fps=29,
)